In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
# 1. PREPARAR LOS DATOS
# -------------------------
def crear_secuencias(datos, n_pasos):
    X, y = [], []
    for i in range(len(datos) - n_pasos):
        X.append(datos[i:i + n_pasos])
        y.append(datos[i + n_pasos])
    return np.array(X), np.array(y)

In [ ]:
# 2. ENTRENAR EL MODELO

def entrenar_rnn(df_train, df_test, n_pasos=24):
    X_train, y_train = crear_secuencias(df_train['value_scaled'].values, n_pasos)
    X_test, y_test = crear_secuencias(df_test['value_scaled'].values, n_pasos)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model = Sequential([
        SimpleRNN(50, activation='tanh', input_shape=(n_pasos, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    print("\n Modelo entrenado.")
    return model, history, X_test, y_test

In [ ]:
# 3. VISUALIZAR LOSS

def graficar_loss(history):
    df_loss = pd.DataFrame({
        'epoch': range(1, len(history.history['loss']) + 1),
        'train_loss': history.history['loss'],
        'val_loss': history.history['val_loss']
    })
    fig = px.line(df_loss, x='epoch', y=['train_loss', 'val_loss'],
                  labels={'value': 'Loss (MSE)', 'epoch': 'Época'},
                  title='Pérdida del entrenamiento RNN')
    fig.show()

In [ ]:
# 4. PREDICCION ONE-STEP

def one_step_prediction(model, X_test, y_test, scaler_path='scaler.pkl'):
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    pred_scaled = model.predict(X_test)
    pred = scaler.inverse_transform(pred_scaled)
    y_real = scaler.inverse_transform(y_test.reshape(-1, 1))

    df_pred = pd.DataFrame({
        'Real': y_real.flatten(),
        'Predicción': pred.flatten()
    })

    fig = px.line(df_pred.head(200), title="Predicción vs Real (One-Step)")
    fig.show()

    mse = mean_squared_error(y_real, pred)
    print(f"\n MSE de One-Step Prediction: {mse:.2f}")


In [ ]:
from preprocesamiento_datos import *
df = pd.read_csv("demanda.csv", parse_dates=['datetime'])
df = imputar_datos_estacionales(df)
df = escalar_datos(df)
df_train, df_test = dividir_train_test(df)

model, history, X_test, y_test = entrenar_rnn(df_train, df_test)
graficar_loss(history)
one_step_prediction(model, X_test, y_test)